In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# --- 1. ЗАГРУЗКА ---
FILE_PATH = '/kaggle/input/train-csv/train.csv'
df = pd.read_csv(FILE_PATH)

# Удаляем пустые строки
df = df.dropna(subset=['review_text', 'reason', 'business_line'])

# --- 2. ГРУППИРОВКА КЛАССОВ ---
synonyms = {
    "неверная консультация": "консультация",
    "некорректная консультация": "консультация",
    "консультация по услуге": "консультация",
    "консультация": "консультация",
    "блокировка/разблокировка карты": "блокировка",
    "ограничение операций": "блокировка",
    "запрет операций": "блокировка",
    "заблокировать сим-карту/разблокировать сим-карту": "блокировка",
    "документы": "документы",
    "запрос документов aml": "документы",
    "предоставление документов": "документы",
    "тарифы": "тарифы и условия",
    "тарифы по депозитным продуктам": "тарифы и условия",
    "тарификация мобайл": "тарифы и условия",
    "акции": "акции и бонусы",
    "кэшбэк": "акции и бонусы",
    "программы лояльности": "акции и бонусы",
    "бонус не начислен": "акции и бонусы",
    "долго решали вопрос": "качество обслуживания",
    "равнодушие": "качество обслуживания",
    "манера общения": "качество обслуживания",
    "претензия на работу доп услуг": "качество обслуживания",
    "общая информация": "другое",
    "другое": "другое"
}
df['reason_clean'] = df['reason'].map(lambda x: synonyms.get(x, x))

# Убираем редкие классы
counts = df['reason_clean'].value_counts()
valid_classes = counts[counts >= 5].index
df = df[df['reason_clean'].isin(valid_classes)]

# --- 3. ИНЖЕНЕРИЯ ПРИЗНАКОВ (ГЛАВНОЕ ИЗМЕНЕНИЕ) ---
# Мы склеиваем "business_line" и "review_text"
# Модель увидит: "Продукт: кредитка | Отзыв: списали лишние проценты..."
def combine_text(row):
    product = str(row['business_line']).strip()
    text = str(row['review_text']).strip()
    # Используем разделитель " | " или спецтокен "[SEP]"
    return f"Продукт: {product} | Отзыв: {text}"

df['combined_text'] = df.apply(combine_text, axis=1)

# Проверим, как это выглядит
print("Пример входных данных для модели:")
print(df['combined_text'].iloc[0])
print("-" * 30)

# --- 4. DATASET ---
labels_list = sorted(df['reason_clean'].unique().tolist())
label2id = {l: i for i, l in enumerate(labels_list)}
id2label = {i: l for i, l in enumerate(labels_list)}
df['label'] = df['reason_clean'].map(label2id)

train_df, test_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)
from sklearn.utils import resample

# Функция для балансировки
def balance_dataframe(df, label_col='label'):
    # 1. Находим размер самого большого класса
    max_size = df[label_col].value_counts().max()
    
    lst = [df]
    for class_index, group in df.groupby(label_col):
        # Если класс меньше максимума, размножаем его
        if len(group) < max_size:
            # Размножаем с возвращением (replace=True)
            augmented = resample(
                group, 
                replace=True, 
                n_samples=max_size, 
                random_state=42
            )
            lst.append(augmented)
        else:
            # Если это и так большой класс, берем как есть (или можно тоже resample, но зачем)
            # Чтобы не дублировать оригинал, который уже в lst[0], 
            # тут логика чуть сложнее. Проще пересобрать всё заново:
            pass
            
    # ПЕРЕПИСЫВАЕМ ЛОГИКУ ДЛЯ ЧИСТОТЫ:
    
    df_balanced = pd.DataFrame()
    # Проходим по каждому классу
    for class_index, group in df.groupby(label_col):
        if len(group) < max_size:
             # Редкие классы дублируем до max_size
             group_resampled = resample(group, replace=True, n_samples=max_size, random_state=42)
        else:
             # Частые классы берем как есть (можно чуть урезать, если хотите undersampling)
             group_resampled = group
             
        df_balanced = pd.concat([df_balanced, group_resampled])
        
    return df_balanced

# --- ПРИМЕНЕНИЕ ---
# Важно: Балансируем ТОЛЬКО train. Test трогать нельзя!
print("Размер train до балансировки:", len(train_df))
train_df_balanced = balance_dataframe(train_df, label_col='label')
print("Размер train после балансировки:", len(train_df_balanced))

# Теперь создаем Dataset из сбалансированного фрейма
train_ds = Dataset.from_pandas(train_df_balanced)
# test_ds оставляем как был!
test_ds = Dataset.from_pandas(test_df)

# --- 5. ТОКЕНИЗАЦИЯ ---
model_name = 'BAAI/bge-m3'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    # ВАЖНО: Токенизируем теперь колонку 'combined_text', а не 'review_text'
    return tokenizer(
        examples["combined_text"], 
        truncation=True, 
        max_length=280 # Чуть увеличили длину, так как добавили business_line
    )

encoded_train = train_ds.map(preprocess, batched=True)
encoded_test = test_ds.map(preprocess, batched=True)

Пример входных данных для модели:
Продукт: кредит наличными | Отзыв: ужаснвы сервис и решение вопросов. Одни отписки. 
9.02.25 вношу частично досрочно сумму. Сумма ежемесячного платежа меняется с 38670 до 38580 -то есть кто закончил 2 класса школы знает , что это 90 рублей. платить этот платеж новый 16 платежей . Это значит 90*16=1440 рублей стало по идее меньше для платы. НО последний платеж изменился , как его называют "корректирующий" с 37189.31 до 38898. опять же 1 класс математика, это значии , что мой последний платеж увеличился на 1708.69! . проблема заключается в том, что при изменении ежемесячного платежа я уменьшаю переплату на 1440 -Казалось бы здоров! Но 17 платеж вырос на 1708,69. а это значит, что экономия которая была в 1440 полностью съедается "корректирующии" платежом и я в общей сложность по графику платежей должна 268.69 больше ! Чем было до этого! 
проблема в том, что в чате меня не слышат. 
фраза "лишних процентов вы не заплатите , график корректны, у вас снизилась

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Map:   0%|          | 0/21296 [00:00<?, ? examples/s]

Map:   0%|          | 0/548 [00:00<?, ? examples/s]

In [3]:
import torch
print(torch.__version__)

2.6.0+cu124


In [4]:
!nvidia-smi

Sun Dec  7 08:45:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Предположим, у вас есть список меток из вашего train dataset
# y_train = [0, 1, 0, 0, 0, 1, 2, ...] 

# Вычисляем веса
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

# Конвертируем в тензор и (важно!) указываем тип float32
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

In [ ]:
from transformers import Trainer
from torch import nn

class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Мы передаем веса при инициализации
        self.class_weights = class_weights
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Переопределяем расчет loss, добавляя веса классов.
        """
        labels = inputs.get("labels")
        
        # Прогоняем данные через модель
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # ВАЖНО: Переносим веса на то же устройство (GPU/CPU), где находится модель
        if self.class_weights is not None:
            weights = self.class_weights.to(model.device)
        else:
            weights = None
            
        # Создаем функцию потерь с весами
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        
        # Считаем loss
        # view(-1, self.model.config.num_labels) нужен, если размерности не совпадают,
        # но для классификации обычно достаточно (logits, labels)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import f1_score, accuracy_score
import torch

# Инициализация модели
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels_list),
    id2label=id2label,
    label2id=label2id
)

# Функция метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_weighted": f1_score(labels, predictions, average="weighted")
    }

# Параметры обучения
args = TrainingArguments(
    output_dir="/kaggle/working/bank_model_results", # Папка внутри Kaggle
    learning_rate=2e-5,
    per_device_train_batch_size=8, # T4 должна потянуть 8 или 16
    per_device_eval_batch_size=16,
    num_train_epochs=4,            # Даем модели время подумать
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    fp16=True,                     # Включаем ускорение GPU
    report_to="none"               # Отключаем WandB, чтобы не просил логин
)

trainer = WeightedTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

print("Запуск обучения на GPU...")
trainer.train()

2025-12-07 08:45:53.063168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765097153.258799      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765097153.315663      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_47/2563911152.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Запуск обучения на GPU...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted
1,2.575900,3.508874,0.087591,0.061818
2,0.942400,3.076965,0.264599,0.250805
3,0.324500,3.237145,0.324818,0.297129


In [ ]:
from sklearn.metrics import classification_report

# Получаем предсказания на тесте
preds = trainer.predict(encoded_test)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

# Переводим числа обратно в текст
y_pred_labels = [id2label[i] for i in y_pred]
y_true_labels = [id2label[i] for i in y_true]

print("\n--- ИТОГОВЫЙ ОТЧЕТ ---")
# Выводим только классы, которые есть в тесте, чтобы избежать warnings
unique_labels = sorted(list(set(y_true_labels)))
print(classification_report(y_true_labels, y_pred_labels, labels=unique_labels))

# Сохраняем готовую модель
save_path = "/kaggle/working/final_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f"Модель сохранена в {save_path}. Вы можете скачать её в разделе Output справа.")

In [ ]:

import shutil
shutil.make_archive("/kaggle/working/my_bank_model", 'zip', "/kaggle/working/final_model")
print("Архив создан! Скачайте my_bank_model.zip из панели Output.")